## 1 Dropout：理论模型
在讲解MLP时我们给出了如下图所示的带有隐藏层的神经网络：
![avatar](../resource/mlp.svg)
其中，对于单个样本$\big([x_1, ..., x_4]^\top, y\big)$，隐藏单元$h_i$的计算表达式为
$$
h_i = \phi \Big(\vec{x}^\top W_h(:,i) + b_h(i)\Big)
$$

若对该隐藏层使用dropout，则该层的每个隐藏单元有一定概率会被丢弃掉。设丢弃概率（超参数）为$p$，则$\forall i, h_i$有$p$的概率会被清零，有$1-p$的概率会被做拉伸。用数学语言描述即
$$
h'_i = \frac{\xi_i}{1-p}h_i,
$$
其中$\xi_i$是一个随机变量，$p(\xi_i = 0) = p$，$p(\xi_i=1) = 1-p$。
则
$$
\mathbb{E} [h'_i] = h_i.
$$
这意味着**dropout不改变输入的期望输出**（这就是要除以$1-p$的原因）。

对上述MLP训练的时候使用dropout，一种可能的网络结构如下：
![avatar](../resource/dropout.svg)
此时MLP的输出不依赖$h_2$和$h_5$。由于在训练中隐藏层神经元的丢弃是随机的，即$h_1, ..., h_5$都有可能被清零，输出层的计算无法过度依赖$h_1, ..., h_5$中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。

**Dropout是一种训练时应对过拟合的方法，并未改变网络的结构。当参数训练完毕并用于测试时，任何参数都不会被dropout。**

## 2 Dropout：从零开始实现

In [13]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import my_utils

def dropout(X, drop_prob):
    X = X.float()
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    if keep_prob == 0:
        return torch.zeros_like(X)
    # mask = 0则对应位置的x将会被dropout，注意要除以keep_prob！
    mask = (torch.rand(X.shape) < keep_prob).float()
    return mask * X / keep_prob

# 测试
X = torch.arange(16).view(2, 8)
print(dropout(X, 0))
print(dropout(X, 0.5))
print(dropout(X, 1))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0., 18., 20., 22., 24.,  0., 28., 30.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


### 2.1 定义含有两个隐藏层的MLP模型参数

使用Fashion-MNIST数据集进行训练和预测：

In [14]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256
W1 = torch.tensor(
    np.random.normal(0, 0.01, size=(num_inputs, num_hiddens1)), 
    dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens1, requires_grad=True)
W2 = torch.tensor(
    np.random.normal(0, 0.01, size=(num_hiddens1, num_hiddens2)),
    dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_hiddens2, requires_grad=True)
W3 = torch.tensor(
    np.random.normal(0, 0.01, size=(num_hiddens2, num_outputs)),
    dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs, requires_grad=True)

params = [W1, b1, W2, b2, W3, b3]

### 2.2 定义模型（在两个隐藏层上使用dropout）

In [15]:
drop_prob1, drop_prob2 = 0.2, 0.5

def net(X, is_training=True):
    X = X.view(-1, num_inputs)
    H1 = (torch.matmul(X, W1) + b1).relu()
    if is_training:
        H1 = dropout(H1, drop_prob1)
    H2 = (torch.matmul(H1, W2) + b2).relu()
    if is_training:
        H2 = dropout(H2, drop_prob2)
    return torch.matmul(H2, W3) + b3

### 2.3 定义模型评估函数

In [16]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        if isinstance(net, torch.nn.Module):
            # 如果模型是通过torch来定义的（必然是nn.Module），
            # 则torch会根据net当前是处于评估模式（eval）还是训练模式（train）来进行抉择是否dropout
            # 评估模型自然要进入eval模式，但是别忘了评估结束后切换回来
            net.eval()
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train()
        else:
            # 模型时从零开始实现的
            if('is_training' in net.__code__.co_varnames):
                # 若模型使用了dropout，则要将is_training设置为false
                acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item()
            else:
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()

### 2.4 训练并测试模型

In [17]:
num_epochs, lr, batch_size = 5, 100., 256    # 注意这里的学习旅需要设置得很大
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = my_utils.load_fashion_mnist(batch_size)
my_utils.general_train(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

epoch 1, loss 0.0047, train acc 0.541, test acc 0.721
epoch 2, loss 0.0023, train acc 0.783, test acc 0.742
epoch 3, loss 0.0019, train acc 0.820, test acc 0.772
epoch 4, loss 0.0018, train acc 0.837, test acc 0.824
epoch 5, loss 0.0016, train acc 0.848, test acc 0.844


## 3 Dropout：PyTorch实现（直接使用torch.nn.dropout()）

### 3.1 定义模型

In [19]:
net = nn.Sequential(
    my_utils.FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens1),
    nn.ReLU(),
    nn.Dropout(drop_prob1),
    nn.Linear(num_hiddens1, num_hiddens2),
    nn.ReLU(),
    nn.Dropout(drop_prob2),
    nn.Linear(num_hiddens2, 10))

for params in net.parameters():
    nn.init.normal_(params, mean=0, std=0.01)

### 3.2 训练并测试

In [21]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
my_utils.general_train(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0052, train acc 0.495, test acc 0.681
epoch 2, loss 0.0024, train acc 0.768, test acc 0.799
epoch 3, loss 0.0021, train acc 0.809, test acc 0.805
epoch 4, loss 0.0018, train acc 0.829, test acc 0.825
epoch 5, loss 0.0017, train acc 0.841, test acc 0.831
